In [1]:
import heapq
from collections import defaultdict
from itertools import ifilter

def h(map, location):
    """ Heuristic function for A*. It is admissible and monotonic. """
    return abs(location[0] - map.end[0]) + abs(location[1] - map.end[1])

In [2]:
class Map:
    def __init__(self, map):
        self.rows = len(map)
        self.cols = len(map[0])
        self.start = (0, 0)
        self.end = (self.rows - 1, self.cols - 1)
        self.passable = [[True if map[row][col] == 0 else False for col in range(self.cols)] for row in range(self.rows)]
        self.g_score = defaultdict(lambda: float("inf"))
        self.g_score[self.start] = 1

        self.f_score = defaultdict(lambda: float("inf"))
        self.f_score[self.start] = self.g_score[self.start] + h(self, self.start)

        self.modified = None # location of the modified cell, if any

    def locations(self):
        """ Iterates over the locations in the map"""
        for row in range(self.rows):
            for col in range(self.cols):
                yield (row, col)

    def modify(self, location, value):
        self.modified = location
        self.passable[location[0]][location[1]] = value
        cost_of_changed = self.g_score[location]
        self.g_score[location] = float("inf")
        self.f_score[location] = float("inf")
        
        farther_locations = ifilter(lambda loc: self.g_score[loc] >= cost_of_changed, self.locations())
        for loc in farther_locations:
            self.g_score[loc] = float("inf")
            self.f_score[loc] = float("inf")

    def get_neighbours(self, loc):
        """ Return a list of neighbors for a given location """
        neighbours = []
        if loc[0] > 0:
            neighbours.append((loc[0] - 1, loc[1]))
        if loc[0] < self.rows - 1:
            neighbours.append((loc[0] + 1, loc[1]))
        if loc[1] > 0:
            neighbours.append((loc[0], loc[1] - 1))
        if loc[1] < self.cols - 1:
            neighbours.append((loc[0], loc[1] + 1))
        return neighbours

    def get_passable_neighbours(self, location):
        neigh = self.get_neighbours(location)
        return filter(lambda loc: self.passable[loc[0]][loc[1]], neigh)
        
    def A_star(self):
        # I literally just implemented the pseudocode from wikipedia, using heapq and tried keeping scores where possible after a call to the self.modify method, to avoid startint from scratch
        open_set = []
        if self.modified is None:
            heapq.heappush(open_set, (self.f_score[self.start], self.start))
        else:
            heapq.heappush(open_set, (self.f_score[self.modified], self.modified))
        
        # if it was previously calculated it should be re-explored, but only if interesting.
        for location in self.locations():
            if self.f_score[location] != float("inf"):
                heapq.heappush(open_set, (self.f_score[location], location))

        while open_set:
            _, current_loc = heapq.heappop(open_set)
            if current_loc == self.end:
                return self.g_score[current_loc]
            for neighbor_loc in self.get_passable_neighbours(current_loc):
                tentative_g_score = self.g_score[current_loc] + 1
                if tentative_g_score < self.g_score[neighbor_loc]:
                    self.g_score[neighbor_loc] = tentative_g_score
                    self.f_score[neighbor_loc] = self.g_score[neighbor_loc] + h(self, neighbor_loc)
                    entry = (self.f_score[neighbor_loc], neighbor_loc)
                    if entry not in open_set:
                        heapq.heappush(open_set, entry)
        # No path found
        return float("inf")

In [3]:
def test(map):
    m = Map(map)
    print(m.A_star())
print("Para [[0]]")
test([[0]])
print("Para 2")
test([[0, 0], [0, 0]])
print("Para 3")
test([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
print("Para [[0, 0]]")
test([[0, 0]])
print("Es 7")
test([[0, 1, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 1, 0]])
print("Es 21")
test([
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0], 
        [0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0]
    ])

Para [[0]]
1
Para 2
3
Para 3
5
Para [[0, 0]]
2
Es 7
7
Es 21
21


In [4]:
def is_useful_removal(location, map):
    """ Does making this cell passable create a path or just a dead end? """
    if map.passable[location[0]][location[1]]:
        return False
    passable_neighbours = map.get_passable_neighbours(location)
    if len(passable_neighbours) <= 1:
        return False
    return True

def useful_removals(map):
    useful_removals = ifilter(lambda loc: is_useful_removal(loc, map), map.locations())
    return useful_removals

In [5]:
def solution(map):
    m = Map(map)
    current_min_length = float("inf")

    for removed_cell in useful_removals(m):
        m.modify(removed_cell, True)
        current_length = m.A_star()
        if current_length < current_min_length:
            current_min_length = current_length
        m.modify(removed_cell, False)
    return current_min_length

In [9]:
# %%timeit
print("Es 7")
print(solution([
        [0, 1, 1, 0],
        [0, 0, 0, 1],
        [1, 1, 0, 0],
        [1, 1, 1, 0]
    ]))
print("Es 11")
print(solution([
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0], 
        [0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0]
    ])
)
print("Es 22")
print(solution([
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0], 
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1], 
        [0, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0]
    ])
)

Es 7
7
Es 11
11
Es 22
22
